In [ ]:
## Dependencies
import os
import pandas as pd


In [ ]:
## Save path to data set in a variable
fsm_elements_publications_path = "data/2022_02-07 FSM Publications.csv"

## Use Pandas to read data
fsm_elements_df = pd.read_csv(fsm_elements_publications_path, encoding = "ISO-8859-1", na_values=['NULL'], dtype={'PubMed ID': 'Int64'})

## Print result
# fsm_elements_df.head()

## Review result
# fsm_elements_df.count()
fsm_elements_df.dtypes

In [ ]:
## Save path to data set in a variable
search_result_path = "data/2022_02-07 Homeless Docs.csv"
search_result_path_1 = "data/2022_02-07 Homeless IS Docs.csv"
search_result_path_2 = "data/2022_02-07 IS Docs.csv"
search_result_path_3 = "data/2022_02-07 STD Docs.csv"
search_result_path_4 = "data/2022_02-07 STD IS Docs.csv"

## Use Pandas to read data
# search_result_df = pd.read_csv(search_result_path, encoding = "ISO-8859-1", na_values=['NULL'], dtype={'PubMed Id ': int})
# search_result_df = pd.read_csv(search_result_path_1, encoding = "ISO-8859-1", na_values=['NULL'], dtype={'PubMed Id ': int})
# search_result_df = pd.read_csv(search_result_path_2, encoding = "ISO-8859-1", na_values=['NULL'],  dtype={'PubMed Id ': int})
# search_result_df = pd.read_csv(search_result_path_3, encoding = "ISO-8859-1", na_values=['NULL'], dtype={'PubMed Id ': int})
# search_result_df = pd.read_csv(search_result_path_4, encoding = "ISO-8859-1", na_values=['NULL'], dtype={'PubMed Id ': int})

## Print result
# search_result_df.head()

## Review columns
search_result_df.columns

In [ ]:
## Save search result DOIs into list

search_result_doi_list = search_result_df['DOI'].tolist()
cleaned_search_result_doi_list = [x for x in search_result_doi_list if str(x) != 'nan']

## Print result
print(search_result_doi_list)

## Print result type
# print(type(search_result_doi_list))

In [ ]:
## Save search result PMIDs into list

search_result_pmid_list = search_result_df['Pubmed Id'].tolist()
cleaned_search_result_pmid_list = [int(x) for x in search_result_pmid_list if str(x) != 'nan']

## Print result
print(cleaned_search_result_pmid_list)

## Print result type
# print(type(cleaned_search_result_pmid_list))

In [ ]:
## Save search result PMIDs into list

search_result_ut_list = search_result_df['UT (Unique WOS ID)'].tolist()
cleaned_search_result_ut_list = [x for x in search_result_ut_list if str(x) != 'nan']


## Print result
print(search_result_ut_list)

## Print result type
# print(type(search_result_ut_list))

In [ ]:
## Create new columns to compare search result list to FSM Elements publications

fsm_elements_df["DOI in Search Result"] = fsm_elements_df["DOI"].isin(cleaned_search_result_doi_list).astype(int)
fsm_elements_df["PMID in Search Result"] = fsm_elements_df["PubMed ID"].isin(cleaned_search_result_pmid_list).astype(int)
fsm_elements_df["UT in Search Result"] = fsm_elements_df["Web of Science Article ID"].isin(cleaned_search_result_ut_list).astype(int)
fsm_elements_df.head(20)

In [ ]:
## If any id is in search results, mark in new column called "Any pub in Search Result"

fsm_elements_df["Any pub in Search Result"] = (fsm_elements_df["DOI in Search Result"] | fsm_elements_df["PMID in Search Result"] | fsm_elements_df["UT in Search Result"]).astype(int)
fsm_elements_df.head()

In [ ]:
fsm_elements_df.dtypes

In [ ]:
# df.drop(df.index[df['myvar'] == 'specific_name'], inplace = True)

dropna_df = fsm_elements_df.loc[fsm_elements_df['Any pub in Search Result'] == 1,: ]
dropna_df.head()

dropna_df.dtypes

In [ ]:
## Group by NetID and sum number of pubs from search results

groupby_df = dropna_df.groupby('Net ID').agg({'Any pub in Search Result': 'sum', 
                                                       'Name' : 'min',
                                                       'Position': 'min', 
                                                       'Department': 'min', 
                                                       'School': 'min',
                                                       'Is Current? 0=No': 'min',
                                                       'Is Academic? 0=No' : 'min',
                                                       'DOI':lambda x: list(x),
                                                       'PubMed ID':lambda x: list(x)
                                              
#                                                     list(map(int, x))
                                                      }).reset_index().sort_values(by=['Any pub in Search Result'], ascending=False).rename(columns={"Any pub in Search Result": "Sum of pubs in Search Result"})
groupby_df.head()

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
groupby_df.to_csv("data/expert_finder.csv", index=False, header=True)